# ChatGPT 3.5 fine-tuning

In [12]:
!pip install -q openai tqdm gradio

## Process Custom dataset
### 세법 또는 사업자종류에 관한 세금 종류


In [13]:
import pandas as pd
from tqdm import tqdm

## 훈련시킬 데이터셋 불러오기

In [ ]:
df = pd.read_excel('train.xlsx')
df

## Format

### 대화형 챗봇을 하기 위해서 꼭 전처리

In [15]:
messages = []

for i, row in tqdm(df.iterrows()):
    try:
        message = {"messages": [
            {"role": "system", "content": "ChatSNS is a chatbot really friendly and concise and a helpful assistant."},
            {"role": "user", "content": row['messages']} 
        ]}
        messages.append(message)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        break

print(messages[:])



5it [00:00, 624.45it/s]

[{'messages': [{'role': 'system', 'content': 'ChatSNS is a chatbot really friendly and concise and a helpful assistant.'}, {'role': 'user', 'content': '종합소득세, 부가가치세, 소득세법상 면세사업의 부가가치세, 취득세, 등록세, 자동차세, 재산세, 종합부동산세, 농어촌특별세, 지방소득세'}]}, {'messages': [{'role': 'system', 'content': 'ChatSNS is a chatbot really friendly and concise and a helpful assistant.'}, {'role': 'user', 'content': '법인세, 부가가치세, 소득세법상 면세사업의 부가가치세, 종합부동산세, 취득세, 등록세, 자동차세, 농어촌특별세, 지방소득세'}]}, {'messages': [{'role': 'system', 'content': 'ChatSNS is a chatbot really friendly and concise and a helpful assistant.'}, {'role': 'user', 'content': '종합소득세, 부가가치세, 소득세법상 면세사업의 부가가치세, 취득세, 등록세, 자동차세, 재산세, 종합부동산세, 농어촌특별세, 지방소득세, 외국인투자촉진법에 따른 외국인투자가산세'}]}, {'messages': [{'role': 'system', 'content': 'ChatSNS is a chatbot really friendly and concise and a helpful assistant.'}, {'role': 'user', 'content': '법인세, 부가가치세, 소득세법상 면세사업의 부가가치세, 종합부동산세, 취득세, 등록세, 자동차세, 농어촌특별세, 지방소득세, 기부금세, 고유목적사업준비금세, 임원보수한도초과액가산세'}]}, {'messages': [{'role': 'system

In [16]:
import json

with open("messages.jsonl", "w", encoding="utf-8") as f:
    for entry in messages:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')


In [17]:
# JSONL 파일 생성 함수
def create_jsonl_file(df, file_path):
    with open(file_path, 'w') as f:
        for item in df:
            json.dump({"input": item}, f)
            f.write('\n')

In [18]:
# JSONL 파일 생성
jsonl_file_path = '학습데이터.jsonl'
create_jsonl_file(messages, jsonl_file_path)

In [ ]:
# Api키 생성
import os
import openai

openai.api_key = "openai.api_key"


#### 기본 prompt

In [ ]:
prompt_template = """
사업자 개인 정보:
0. 오늘 날짜 : 2023년 9월 19일
1. 올 해  매출 :
2. 경비 :
3. 자산 : 
4. 세금 전 수익 : 
5. 업종 : 
6. 사업자종류 : 

과세표준   세율   누진공제액
1,200만 원 이하   6%   -
1,200만 원 초과 ~ 4,600만 원 이하   15%   108만 원
4,600만 원 초과 ~ 8,800만 원 이하   24%   522만 원
8,800만 원 초과 ~ 1.5억 원 이하   35%   1,490만 원
1.5억 원 초과 ~ 3억 원 이하   38%   1,940만 원
3억 원 초과 ~ 5억 원 이하   40%   2,540만 원
5억 원 초과 ~ 10억 원 이하   42%   3,540만 원
10억 원 초과   45%   6,540만 원
다음 정보를 참고하여 500 글자 내로 답변해주세요. 답변은 구체적인 금액을 포함해야 합니다. 자세한 내용은 세무사와 상담하라고 해주세요:
"""

In [ ]:
# 파인 튜닝 실행 함수
def fine_tune_model(jsonl_file):
    with open(jsonl_file, 'r') as f:
        training_data_jsonl = f.read()

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "ChatSNS is a chatbot really fridendly and concise and a helpful assistant and Unconditionally, you should never lie.."},
            {"role": "user", "content": "training_data_jsonl"}
        ],
        max_tokens=1000,  # 원하는 최대 출력 길이 설정 (상황에 맞게 조정)
        n=1,  # 생성할 출력 예시 개수 (상황에 맞게 조정)
        temperature=0.2,  # 다양성 조절 (0에서 1 사이 값; 낮으면 보수적, 높으면 창의적)
    )

    for choice in response.choices:
        generated_text = choice.message['content'].strip()
        # 생성된 결과 처리 로직 추가

fine_tune_model(jsonl_file_path)

print(fine_tune_model(jsonl_file_path))

### Upload the file to open AI server

## Test

In [20]:
import gradio as gr

def answer(state, state_chatbot, text):
    messages = state + [{
        'role': 'user',
        'content': text
    }]
 
    res = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages
    )
 
    msg = res['choices'][0]['messages']['content']
 
    new_state = [{
        'role': 'user',
        'content': text
    }, {
        'role': 'assistant',
        'content': msg
    }]
 
    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]
 
    print(state)
 
    return state, state_chatbot, state_chatbot
 
 
with gr.Blocks(css='#chatbot .overflow-y-auto{height:300px}') as demo:
    state = gr.State([{
        'role': 'system',
        'content': 'You are a helpful assistant.'
    }])
    state_chatbot = gr.State([])
 
    with gr.Row():
        gr.HTML("""<div style="text-align: center; max-width: 300px; margin: 0 auto;">
            <div>
                <h1>   영   수   증   </h1>
            </div>
            <p style="margin-bottom: 10px; font-size: 30%">
            </p>
        </div>""")
 
    with gr.Row():
        chatbot = gr.Chatbot(elem_id='무엇을 도와드릴까요?').style(container=False)
 
    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder='사업자를 선택해주세요').style(container=False)
 
    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: '', None, txt)
 
demo.launch(debug=True, share=True)

C:\Users\user\AppData\Local\Temp\ipykernel_16312\3254947575.py:49: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  chatbot = gr.Chatbot(elem_id='무엇을 도와드릴까요?').style(container=False)
C:\Users\user\AppData\Local\Temp\ipykernel_16312\3254947575.py:52: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  txt = gr.Textbox(show_label=False, placeholder='사업자를 선택해주세요').style(container=False)


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\billl\lib\site-packages\gradio\routes.py", line 508, in predict
    output = await route_utils.call_process_api(
  File "c:\Users\user\anaconda3\envs\billl\lib\site-packages\gradio\route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\user\anaconda3\envs\billl\lib\site-packages\gradio\blocks.py", line 1437, in process_api
    result = await self.call_function(
  File "c:\Users\user\anaconda3\envs\billl\lib\site-packages\gradio\blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\user\anaconda3\envs\billl\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\user\anaconda3\envs\billl\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "c:\Users\user\anaconda3\e

Keyboard interruption in main thread... closing server.


In [ ]:
# def answer(state, text):
#     if not state:
#         state = [{
#             "role": "system",
#             "content": "You are a helpful assistant."
#         }]
    
#     messages = state + [{
#         'role': 'user',
#         'content': text
#     }]

#     res = openai.ChatCompletion.create(
#         model='gpt-3.5-turbo',
#         messages=messages
#     )

#     msg = res['choices'][0]['message']['content']

#     new_state = state + [{
#         'role': 'user',
#         'content': text
#     }, {
#         'role': 'assistant',
#         'content': msg
#     }]

#     return new_state, "", [(text, msg)]

In [ ]:
# model_id = "ft:gpt-3.5-turbo-"

# uesr_input = "개인사업자"

# completion = openai.ChatCompletion.create(
#     model = model_id,
#     messages = [
#         {"role": "system", "content": "ChatSNS is a chatbot really fridendly and concise."},
#         {"role": "user", "content": uesr_input},
#     ]
# )

# print(completion.choices[0].message["content"])

# 단순 웹페이지에 챗봇 만들기
## Gradio 사용

In [ ]:
# import gradio as gr

# def predict (input, history):
#   history.append( {"role": "user", "content": input})
#   gpt_response = openai.ChatCompletion.create(
#       model = model_id,
#       messages = history
#   )

#   response = gpt_response[ "choices"] [0][][ "message" ][ "content "]

#   history.append ({"role": "assistant", "content": response})

#   messages = [(history[i]["content"], history[i+1]["content"J]) for i in range(1, len(history), 2)]

#   return messages, history


# with gr.Blocks() as demo:
#   chatbot = gr.Chatbot(label= "ChatBot")

#   state = gr.State([{
#       "role": "system",
#       "content": "ChatSNS is a chatbot really friendly and concise."
#   }])

#   with gr.Row():
#     txt = gr.Textbox(show_label =False, placeholder = "운영하시는 사업자를 선택해주세요.").style(container = False)

#   txt.submit(predict, [txt, style], [chatbot, state])

# demo.launch(debug = True, share = )